In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
import sys
from os.path import join
from functools import reduce
import torch.nn as nn
from torch.nn.functional import pad as torch_pad

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))
from learning_rate_search import *

In [3]:
#export
def init_conv_weight(shape, leak=0.):
    # default to he init
    assert(shape[2] == shape[3])
    # in channel * receptive field (kernel area)
    fan = shape[1] * shape[2] * shape[3]
    gain_sq = 2.0 / (1 + leak**2)
    return torch.randn(*shape) * (gain_sq / fan)**0.5

def zero_pad(inp, pad):
    return torch_pad(inp, [pad]*4, 'constant', 0)

In [4]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train = x_train[:100].view(-1, 1, 28, 28)
torch_conv = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3)

In [5]:
torch_conv_out_mean = 0.
torch_conv_out_std = 0.
for _ in range(100):
    torch_conv.weight = nn.Parameter(init_conv_weight((4, 1, 3, 3)))
    torch_conv_out = torch_conv(x_train).clamp_min(0.)
    torch_conv_out_mean += torch_conv_out.mean()
    torch_conv_out_std += torch_conv_out.std()
torch_conv_out_mean /= 100.0
torch_conv_out_std /= 100.0
print(f'mean: {torch_conv_out_mean}, std: {torch_conv_out_std}')

mean: 0.47612112760543823, std: 0.9395545721054077


In [6]:
#export
class Reshape(Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape
    
    def fwd(self, inp): 
        return inp.view(-1, *self.shape)
    
    def bwd(self, out, inp): 
        # simply reverse the fwd
        inp.g = out.g.reshape(-1, reduce(lambda x,y: x*y, self.shape))
    
    def __repr__(self): 
        return f'Reshape{self.shape}'

In [7]:
#export
class Flatten(Module):
    def __init__(self):
        super().__init__()
        
    def fwd(self, inp):
        self.batch_size, *self.shape = inp.shape
        return inp.view(self.batch_size, -1)
    
    def bwd(self, out, inp):
        inp.g = out.g.view(-1, *self.shape)
        
    def __repr__(self):
        return 'Flatten()'

In [8]:
#export
class Conv(Module):
    def __init__(self, c_in, c_out, k_s, stride=1, pad=0, leak=1.):
        super().__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.k_s = k_s
        self.stride = stride
        self.pad = pad   
        
        self.w = Parameter(init_conv_weight((c_out, c_in, k_s, k_s), leak))
        self.b = Parameter(torch.zeros(c_out))
    
    def fwd(self, inp):
        batch_size, _, in_h, in_w = inp.shape
        inp = zero_pad(inp, self.pad)
        _, _, p_h, p_w = inp.shape

        # init output
        out_dim = lambda d: (d + 2 * self.pad - self.k_s) // self.stride + 1
        out = torch.zeros(batch_size, self.c_out, out_dim(in_h), out_dim(in_w))

        # compute output cell by cell
        for i in range(0, p_h - self.k_s + 1, self.stride):
            for j in range(0, p_w - self.k_s + 1, self.stride):
                receptive_field = inp[:, :, i:i+self.k_s, j:j+self.k_s].unsqueeze(1)
                out[:, :, i//self.stride, j//self.stride] = (receptive_field * self.w.data).sum((-1,-2,-3)) + self.b.data

        return out
    
    def bwd(self, out, inp):
        # source of var names and math calcs: https://medium.com/@pavisj/convolutions-and-backpropagations-46026a8f5d2c
        dL = out.g
        X, F, B = zero_pad(inp, self.pad), self.w.data, self.b.data
        dX, dF, dB = torch.zeros_like(X), torch.zeros_like(F), torch.zeros_like(B)
        k_s = F.shape[2]
        _, _, out_h, out_w = dL.shape

        # each cell in output are computed from a receptive field in input
        for i in range(out_h):
            for j in range(out_w):
                i_s, j_s = i * self.stride, j * self.stride
                receptive_field = X[:, :, j_s: j_s+k_s, i_s: i_s+k_s].unsqueeze(1)
                dL_section = dL[:, :, j, i][..., None, None, None]

                dX[:, :, j_s: j_s+k_s, i_s: i_s+k_s] += (F * dL_section).sum(1)
                dF += (receptive_field * dL_section).sum(0)
                dB += dL[:, :, j, i].sum(0)

        self.w.update(dF)
        self.b.update(dB)
        inp.g = dX if pad == 0 else dX[:, :, self.pad: -self.pad, self.pad: -self.pad]
    
    def __repr__(self): return f'Conv(in: {self.c_in}, out: {self.c_out}, kernel: {self.k_s}, stride: {self.stride}, pad: {self.pad})'

In [9]:
inp = torch.randn(12, 18, 37, 37)
batch_size = inp.shape[0]
c_in, c_out = inp.shape[1], 7
k_s = 5
pad = 3
stride = 3

print(f'input shape: {tuple(inp.shape)}')
print(f'padding: {pad}')
print(f'stride: {stride}')

input shape: (12, 18, 37, 37)
padding: 3
stride: 3


In [10]:
torch_conv = nn.Conv2d(c_in, c_out, k_s, stride, pad)
torch_res = torch_conv(inp)

conv_layer = Conv(c_in, c_out, k_s, stride, pad)
conv_layer.w = Parameter(torch_conv.weight)
conv_layer.b = Parameter(torch_conv.bias)
my_res = conv_layer.fwd(inp)

print(f'weight shape {tuple(conv_layer.w.data.shape)}')
print(f'bias shape: {tuple(conv_layer.b.data.shape)}')
print(f'output shape: {tuple(my_res.shape)}')

assert(my_res.shape == torch_res.shape)
test_near(my_res, torch_res)

weight shape (7, 18, 5, 5)
bias shape: (7,)
output shape: (12, 7, 13, 13)


In [11]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train, y_train = x_train[:32], y_train[:32]

In [12]:
model = Sequential(Reshape((1, 28, 28)),
                   Conv(1, 8, 5, stride=4, pad=2, leak=0.), # 8, 7, 7 
                   ReLU(), 
                   Conv(8, 16, 3, stride=2, pad=1, leak=0.), # 16, 4, 4
                   Flatten(),
                   Linear(256, 10, True))
loss_fn = CrossEntropy()

In [13]:
model

(Sequential)
	(Layer1) Reshape(1, 28, 28)
	(Layer2) Conv(in: 1, out: 8, kernel: 5, stride: 4, pad: 2)
	(Layer3) ReLU()
	(Layer4) Conv(in: 8, out: 16, kernel: 3, stride: 2, pad: 1)
	(Layer5) Flatten()
	(Layer6) Linear(256, 10)

In [14]:
loss = loss_fn(model(x_train), y_train)
loss_fn.backward()
model.backward()

xtg = x_train.g.clone()
w1g = model.layers[1].w.grad.clone()
b1g = model.layers[1].b.grad.clone()
w2g = model.layers[3].w.grad.clone()
b2g = model.layers[3].b.grad.clone()
w3g = model.layers[5].w.grad.clone()
b3g = model.layers[5].b.grad.clone()

x_train2 = x_train.clone().requires_grad_(True)
model.layers[1].w.data.requires_grad_(True)
model.layers[1].b.data.requires_grad_(True)
model.layers[3].w.data.requires_grad_(True)
model.layers[3].b.data.requires_grad_(True)
model.layers[5].w.data.requires_grad_(True)
model.layers[5].b.data.requires_grad_(True)

loss = loss_fn(model(x_train2), y_train)
loss.backward()

test_near(w1g, model.layers[1].w.data.grad)
test_near(b1g, model.layers[1].b.data.grad)
test_near(w2g, model.layers[3].w.data.grad)
test_near(b2g, model.layers[3].b.data.grad)
test_near(w3g, model.layers[5].w.data.grad)
test_near(b3g, model.layers[5].b.data.grad)

In [15]:
num_hidden = 50
batch_size = 64
num_epochs = 20
learning_rate = 0.1

x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train, y_train, x_valid, y_valid = x_train[:8000], y_train[:8000], x_valid[:2000], y_valid[:2000]

data_bunch = get_data_bunch(x_train, y_train, x_valid, y_valid, batch_size)
model = Sequential(Reshape((1, 28, 28)),
                   Conv(1, 8, 5, stride=4, pad=2, leak=0.), # 8, 7, 7 
                   ReLU(), 
                   Conv(8, 16, 3, stride=2, pad=1, leak=0.), # 16, 4, 4
                   Flatten(),
                   Linear(256, 10, True))
optimizer = Optimizer(list(model.parameters()), learning_rate=learning_rate)
loss_fn = CrossEntropy()
learner = Learner(model, optimizer, loss_fn, data_bunch)

In [16]:
runner = Runner(learner, [StatsLogging([compute_accuracy])])
print(runner)

(DataBunch) 
	(DataLoader) 
		(Dataset) x: (8000, 784), y: (8000,)
		(Sampler) total: 8000, batch_size: 64, shuffle: True
	(DataLoader) 
		(Dataset) x: (2000, 784), y: (2000,)
		(Sampler) total: 2000, batch_size: 128, shuffle: False
(Sequential)
	(Layer1) Reshape(1, 28, 28)
	(Layer2) Conv(in: 1, out: 8, kernel: 5, stride: 4, pad: 2)
	(Layer3) ReLU()
	(Layer4) Conv(in: 8, out: 16, kernel: 3, stride: 2, pad: 1)
	(Layer5) Flatten()
	(Layer6) Linear(256, 10)
(CrossEntropy)
(Optimizer) num_params: 6, hyper_params: ['learning_rate']
(Callbacks) TrainEval StatsLogging


In [17]:
runner.fit(5)

Epoch - 1
train metrics - [0.0023859403133392335, 0.793625]
valid metrics - [0.02583139419555664, 0.8815]

Epoch - 2
train metrics - [0.00431155252456665, 0.9125]
valid metrics - [0.02160742950439453, 0.904]

Epoch - 3
train metrics - [0.0009750940799713135, 0.92975]
valid metrics - [0.018961414337158203, 0.9195]

Epoch - 4
train metrics - [0.0018258848190307617, 0.943]
valid metrics - [0.018419771194458007, 0.935]

Epoch - 5
train metrics - [0.0006376029253005981, 0.94775]
valid metrics - [0.01726538848876953, 0.939]

